# Fixed point optimization of a cubic function

In this lab, we implement a fixed-point version of a monic cubic function:

```python
   y = x**3 + a[2]*x**2 + a[1]*x + a[0]
```

We will follow the similar sequence of steps as the [fixed point demo](https://sdrangan.github.io/hwdesign/docs/demos/fixp/).

## Floating Point Version

We begin by writing a floating point version of the function.

In [ ]:
import numpy as np
from numpy.typing import NDArray

def cubic_float(
    x: NDArray[np.float64],  # shape (n,)
    a: NDArray[np.float64],  # shape (n, 3)
) -> NDArray[np.float64]:
    """
    Evaluate a monic cubic polynomial at points x:

        y = a[:,0] + a[:,1] * x + a[:,2] * x**2 +  x**3
    """
    y = a[:,0] + a[:,1] * x + a[:,2] * x**2 + x**3
    return y

To illustrate, we plot the function for some values:


In [ ]:
nplot = 100
xplot = np.linspace(-4.0, 4.0, nplot)
aplot = np.array([2.0, -0.5, -3.0])
aplot = aplot[None,:].repeat(nplot, axis=0)

yplot = cubic_float(xplot, aplot)
import matplotlib.pyplot as plt
plt.plot(xplot, yplot)
plt.title("Monic Cubic Polynomial")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.show()

## Create a Fixed Point Version

We now create a fixed point version. THere are several ways to handle overflow.
One way is to saturate after each product.  For example, suppose that `xint`
is the integer representation of `x` in `Q(wid,fbits)`.  We can implement the 
product such as `xint*xint`, as follows:

```python
x2int = saturate((xint * xint) >> fbits, wid)
```

This product `(xint * xint) >> fbits` will be in `Q(wid2, fbits)` where 
`wid2= 2*wid-fbits`.   We then saturate each product back to `wid` bits.

Similarly, we can use:

```python
x3int = saturate((xint * x2int) >> fbits, wid_acc)
```

to get the cubic term.  Use this pattern to complete the following code.

In [ ]:
import sys
if "google.colab" in sys.modules:
    !pip install git+https://github.com/sdrangan/hwdesign.git

from xilinxutils.fixputils import truncate, saturate

In [ ]:
def cubic_fixed(
        xint : NDArray[np.int64],  # shape (n,)
        aint : NDArray[np.int64],  # shape (n, 4)
        wid : int = 16,   # total bit width
        fbits : int = 8
    ) -> NDArray[np.int64]:
    """
    Evaluate a monic cubic polynomial in fixed point:
        y = a[:,0] + a[:,1] * x + a[:,2] * x**2 + x**3


    Parameters
    ----------
    xint : NDArray[np.int64]
        Input values in Q(wid, fbits), shape (n,)
    aint : NDArray[np.int64]
        Coefficients in Q(wid, fbits), shape (n, deg+1) where deg=3
    wid : int
        Total bit width of inputs and outputs
    fbits : int
        Number of fractional bits in inputs and outputs
    
        
    Returns
    -------
    yint : NDArray[np.int64]
        Output values in Q(wid, fbits), shape (n,)
    """

    # TODO: Implement fixed-point cubic evaluation.
    # Use the saturate function after each product and sum to ensure
    # that intermediate values do not exceed the specified bit width.
    # Hint: You will need to compute the following intermediate values:
    #   x2int = ...
    #   x3int = ...
    #   yint = ...
    # Compute terms in Q(wid_acc, fbits)
    

    return yint



    

Now plot a comparison of the the floating and fixed point function over the set of values xplot.
This part will not be submitted, but it will help you make sure that your algorithm is working correctly.
You should see:

- For `wid=16`, `fbits=8`, the floating and fixed point versions match well over this range of inputs
- For `wid=16`, `fbits=12`, the fixed point version clearly saturates


In [ ]:
# Bit widths to test
wid_plot = [16, 16]
fbits_plot = [8, 12]
nwid = len(wid_plot)


plt.subplots(1, nwid,figsize=(4*nwid, 4))

for i in range(nwid):

    # TODO: Compute fixed-point outputs for this bit width
    #   xplot_int = ...
    #   aplot_int = ...
    #   yplot_int = ...
    #   yplot_fix = ...
    


    # TODO:  Plot the fixed and floating point results in a subplot
    #  plt.subplot(...)
    #  plt.plot(...)
    

for i in range(nwid):
    w = wid_plot[i]
    f = fbits_plot[i]
    
    # 1. 转换输入为定点整数
    xplot_int = np.round(xplot * (2**f)).astype(np.int64)
    aplot_int = np.round(aplot * (2**f)).astype(np.int64)
    
    # 2. 调用你写的函数
    yplot_int = cubic_fixed(xplot_int, aplot_int, wid=w, fbits=f)
    
    # 3. 转回浮点数用于绘图比较
    yplot_fix = yplot_int.astype(np.float64) / (2**f)
    
    # 4. 绘图
    plt.subplot(1, nwid, i+1)
    plt.plot(xplot, yplot, label='Float')
    plt.plot(xplot, yplot_fix, '--', label=f'Fixed Q{w}.{f}')
    plt.title(f"W={w}, F={f}")
    plt.legend()
    plt.grid(True)

## Generate Test Vectors

As in the demo, you could optimize the bit widths.  But, we skip this step.  Here we just generate test vectors.  We will generate two test vectors:

- `test_vectors/tv_w16_f8.csv` for `wid=16` and `fbits=8`.  In this test case, there are no overflows.
This test case will ensure that the regular logic in the SV implmentation matches the python implementation.
- `test_vectors/tv_w16_f12.csv` for `wid=16` and `fbits=12`.  In this test case, there is significant overflow.  
The test case will good to test the SV implementation performs overflow exactly in a manner that the python implementation does.

The test also measures the MSE:  You should see approximately the following values.
Specifically, with `fbits=8`, you get a very good approximation, but with `fbits=12`,
the logic begins to saturate.

| W (Total Bits) | F (Fraction Bits) | Expected MSE      |
|----------------|-------------------|-------------------|
| 16             | 8                 | ~1e-5       |
| 16             | 12                | ~0.2              |



In [ ]:
import os
import pandas as pd

# Create test_vectors directory if it doesn't exist
os.makedirs('test_outputs', exist_ok=True)

# Generate random samples
nsamp = 100
deg = 3
a = np.random.uniform(-4.0, 4.0, (nsamp, deg))
x = np.random.uniform(-2.0, 2.0, nsamp)

# Fix parameters for test vector generation
wid_acc = 32 
wid = 16
fbits_test = [8, 12]

for fbits in fbits_test:
    print(f'\nGenerating test vectors for W={wid}, F={fbits}')

    # TODO: Generate test vectors
    #   y = floating point output
    #   xint = fixed point input x
    #   aint = fixed point input a
    #   yint = fixed point output
    #   yfix = fixed point output back converted to float

    # Compute floating point output
    y = cubic_float(x, a)

    # Convert to fixed point representation of inputs
    xint = np.round(x * (2**fbits)).astype(np.int64)
    aint = np.round(a * (2**fbits)).astype(np.int64)

    # Compute fixed point outputs
    yint = cubic_fixed(
        xint=xint, 
        aint=aint, 
        wid=wid, 
        fbits=fbits)
    yfix = yint.astype(np.float64) / (2**fbits)

    mse = np.mean((y - yfix)**2)/ np.mean(y**2)
    print("Mean squared error: %12.4e" % mse)

    # Create DataFrame with all values
    df = pd.DataFrame({
        'xint': xint,
        'aint0': aint[:, 0],
        'aint1': aint[:, 1],
        'aint2': aint[:, 2],
        'y': y,
        'yint': yint,
        'yfix': yfix
    })

    # Save to CSV file
    csv_filename = f'test_outputs/tv_w{wid}_f{fbits}.csv'
    df.to_csv(csv_filename, index=False)
    print(f"Test vectors saved to {csv_filename}")
    print(f"Number of test vectors: {len(df)}")
    print(f"\nFirst 5 rows:")
    print(df.head())
